In [13]:
#!/usr/bin/env python

# Copyright 2016, Rohan Dandage <rraadd_8@hotmail.com>
# This program is distributed under General Public License v. 3.    

import sys
import os
from os.path import exists,splitext,basename
import glob
import nd2reader
import pandas as pd
import string
import numpy as np
from scipy import stats,ndimage
from multiprocessing import Pool
import cv2
from skimage.segmentation import random_walker
# from skimage.data import binary_blobs
from skimage import io,exposure,restoration,filters,morphology,measure
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import subprocess
from nd2kins import nd2arr_list,raw2phasecorr,arr_list2regions
from nd2vid import arr_list2vid
from nd2vid import main as nd2vid

In [11]:
fh_xls='/home/kclabws1/Documents/propro/writ/prjs/2_chem_chap_screens/data/yeast_gfp_half_life/data/160309_sdsd_replicates/160309_sdsd_replicates.xlsx'
well='C03'
info=pd.read_excel(fh_xls,'info')
info=info.set_index('varname')
for var in info.iterrows() :
    val=info['input'][var[0]]
    if not pd.isnull(val):
        exec("%s=info['input']['%s']" % (var[0],var[0]),locals(), globals())
    else:
        exec("%s=info['default']['%s']" % (var[0],var[0]),locals(), globals())
data_job=pd.read_excel(fh_xls,'JobView')
data_fns=pd.pivot_table(data_job,values='File Name',index='Loop_bleach Index',columns='Well Name', aggfunc=lambda x: x.iloc[0])
data_fns_P =pd.pivot_table(data_job,values='File Name',index='TimeLapse1 Index',columns='Well Name', aggfunc=lambda x: x.iloc[0])
data_fns   =pd.concat([data_fns,data_fns_P],axis=0)

In [3]:
nd2vid(fh_xls,well)

>>> STATUS  : nd2vid : C03


/home/kclabws1/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/skimage/util/dtype.py:111: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [14]:
# nd_fns=data_fns[well].dropna().unique()
# arr_list=nd2arr_list(nd_dh,nd_fns)
# arr_list_stb=raw2phasecorr(arr_list)
# regions,kins_mean=arr_list2regions(arr_list_stb,3)
arr_list2vid(arr_list_stb,regions,kins_mean,('%s.%sstb.mp4' % (fh_xls,well)),384, 384)

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# from skimage import data, img_as_float,filters
# from skimage.restoration import denoise_tv_chambolle, denoise_bilateral

# %pylab inline
# pre_bleach=arr_list_stb[0]
pre_bleach=pre_bleach_A16
denoised=restoration.denoise_bilateral(pre_bleach.astype('uint16'), sigma_range=0.01, sigma_spatial=15)
smoothened = filters.median(denoised,np.ones((4,4)))
markers = np.zeros(smoothened.shape, dtype=np.uint)
markers[smoothened < filters.threshold_otsu(smoothened)] = 1
markers[smoothened > filters.threshold_otsu(smoothened)] = 2
labels = random_walker(smoothened, markers, beta=10, mode='bf')
regions= measure.label(labels)
props = measure.regionprops(regions,arr_list[0]) #ref
regions_areas=np.array([prop.area  for prop in props])
regions_means=np.array([prop.mean_intensity  for prop in props])
regions_lbls =np.array([prop.label for prop in props])
regions_index_large=np.where((regions_areas<10000) & (regions_areas>200) & (regions_means>3000))[0]
regions_lbls_large=regions_lbls[regions_index_large] 
regions_large=np.zeros((regions.shape), dtype=bool)
for i in regions_lbls_large:
    booli = (regions == i) 
    regions_large=np.logical_or(regions_large,booli)
plt.figure(figsize=(18, 5))
ax1=plt.subplot(1,4,1)
ax1.imshow(pre_bleach)
ax2=plt.subplot(1,4,2)
ax2.imshow(smoothened)
ax3=plt.subplot(1,4,3)
ax3.imshow(regions,cmap='spectral')
ax4=plt.subplot(1,4,4)
ax4.imshow(regions_large,cmap='spectral')

In [ ]:
#random walker
pre_bleach=arr_list_stb[0]
smoothened = filters.median(pre_bleach.astype('uint16'),np.ones((4,4)))
markers = np.zeros(smoothened.shape, dtype=np.uint)
markers[smoothened < filters.threshold_otsu(smoothened)] = 1
markers[smoothened > filters.threshold_otsu(smoothened)] = 2
labels = random_walker(smoothened, markers, beta=10, mode='bf')
regions= measure.label(labels)
# plt.imshow(regions,cmap='jet')
label_objects, nb_labels = ndi.label(regions)
sizes = np.bincount(label_objects.ravel())
mask_sizes = sizes > 100
mask_sizes[0] = 0
regions_cleaned = mask_sizes[label_objects]

In [ ]:
#watershed
from skimage.filters import sobel
from skimage import img_as_float
# pre_bleach_float = img_as_float(pre_bleach.astype('float64'))
image=pre_bleach.astype('float64')
elevation_map = sobel(image)
smoothened = filters.median(pre_bleach.astype('uint16'),np.ones((4,4)))
markers = np.zeros_like(image)
markers[smoothened < filters.threshold_otsu(smoothened)] = 1
markers[smoothened > filters.threshold_otsu(smoothened)] = 2
segmentation = morphology.watershed(elevation_map, markers)
plt.imshow(segmentation, cmap=plt.cm.spectral, interpolation='nearest')